In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
import time
import requests
from urllib.parse import quote
from urllib.error import HTTPError
import urllib.request

from selenium.webdriver.chrome.options import Options
from selenium import webdriver
import string

import psutil
import os

from time import sleep

prefix = "https://www.liepin.com"

### 获取当前所有岗位的连接

In [6]:
url = "https://www.liepin.com/zhaopin/?init=-1&headckid=3e053079b29e4ce9&flushckid=1&fromSearchBtn=2&dqs=280020&pubTime=1&ckid=5e2017eff8c8fe62&sfrom=click-pc_homepage-centre_searchbox-search_new&key=%E4%BC%9A%E8%AE%A1&siTag=yBAlj2NjO-bNRNh2rd-zhw%7EDARaeHgTI7JY9N3sNhM1Ow&d_sfrom=search_fp&d_ckId=dd21085024abcfea7a31cf5ad246d1da&d_curPage=0&d_pageSize=40&d_headId=42d81e285bf9fc398e27eb711e0ea871"
list_job_link = []

while (True):
    try:
        html = urlopen(url)
    except UnicodeEncodeError as e:
        print ("TOP exception " + url)
        url = quote(url, safe=string.printable)
        html = urlopen(url)
    if html.headers.get_content_charset() is None:
        bs = BeautifulSoup(html.read(), 'html.parser')
    else:
        try:
            bs = BeautifulSoup(html.read().decode(html.headers.get_content_charset()), 'html.parser')
        except UnicodeDecodeError as e:
            bs = BeautifulSoup(html.read(), 'html.parser')
            print("exception ---> " + url)
        
    job_object_list = bs.find_all('a', href=re.compile('https://www.liepin.com/job/[0-9]+.*'))
    for current_job_object in job_object_list:
        list_job_link.append(current_job_object.attrs['href'])

    next_list_job_page = bs.find_all('a', text='下一页')
    if len(next_list_job_page) > 0:
        url = prefix + "" + next_list_job_page[0].attrs['href']
    else:
        break

TOP exception https://www.liepin.com/zhaopin/?init=-1&headckid=3e053079b29e4ce9&fromSearchBtn=2&dqs=280020&pubTime=1&ckid=dc6d3affa088811d°radeFlag=0&sfrom=click-pc_homepage-centre_searchbox-search_new&key=%E4%BC%9A%E8%AE%A1&siTag=yBAlj2NjO-bNRNh2rd-zhw%7E8mvd7eAE10Pm4Y1OXPJvjA&d_sfrom=search_fp&d_ckId=b59b1c60adaf5291b0e7c3a8b77f3103&d_curPage=0&d_pageSize=40&d_headId=42d81e285bf9fc398e27eb711e0ea871&curPage=1
TOP exception https://www.liepin.com/zhaopin/?init=-1&headckid=3e053079b29e4ce9&fromSearchBtn=2&dqs=280020&pubTime=1&ckid=dc6d3affa088811d%EF%BF%BDradeFlag%3D0°radeFlag=0&sfrom=click-pc_homepage-centre_searchbox-search_new&key=%E4%BC%9A%E8%AE%A1&siTag=yBAlj2NjO-bNRNh2rd-zhw%7E8mvd7eAE10Pm4Y1OXPJvjA&d_sfrom=search_fp&d_ckId=b59b1c60adaf5291b0e7c3a8b77f3103&d_curPage=1&d_pageSize=40&d_headId=42d81e285bf9fc398e27eb711e0ea871&curPage=2
TOP exception https://www.liepin.com/zhaopin/?init=-1&headckid=3e053079b29e4ce9&fromSearchBtn=2&dqs=280020&pubTime=1&ckid=dc6d3affa088811d%EF%BF%BDra

InvalidURL: nonnumeric port: ';'

In [48]:
current_job_link = list_job_link[0]
print(current_job_link)
sub_html = urlopen(current_job_link)
sub_bs = BeautifulSoup(sub_html.read().decode(sub_html.headers.get_content_charset()), 'html.parser')

https://www.liepin.com/job/1923223061.shtml


In [9]:
def find_first_element(inside_bs, target_tag, condition, target_attribut, return_original=False):
    target_data = inside_bs.find_all(target_tag, condition)
    if len(target_data) > 0:
        target_data = target_data[0]
        if return_original == True:
            return target_data
        if target_attribut == 'text':
            return target_data.get_text()
        else:
            return target_data.attrs[target_attribut]
    else:
        return ''
    
def find_regular_expression(string, pattern, group):
    string_pattern = re.compile(pattern)
    result = re.match(string_pattern, string)
    if result is None:
        return ''
    else:
        return result.group(group)
    
def find_place(location, city=True):
    if(location==''):
        return ''
    else:
        string_pattern = '^(.+)-(.+)$'
        result = re.match(string_pattern, location)
        if result is None:
            if city==True:
                return location
            else:
                return ''
        else:
            if city==True:
                return result.group(1)
            else:
                return result.group(2)
        
def find_salary(salaries, unit=False, min_value=True):
    salary_pattern = '^([0-9]+)-([0-9]+)([万千]+).+'
    position = 0
    if unit == True:
        position = 3
    else:
        if min_value == True:
            position = 1
        else:
            position = 2
    return find_regular_expression(salaries, salary_pattern, position)

def delete_chromedriver():
    pids = psutil.pids()
    for pid in pids:
            p = psutil.Process(pid)
            if p.name() == 'chromedriver.exe':
                cmd = 'taskkill /f /PID '+str(pid)
                os.system(cmd)
            if p.name() == 'geckodriver.exe':
                cmd = 'taskkill /f /PID '+str(pid)
                os.system(cmd)

In [12]:
output_file = "C:\\Users\\hao.ren3\\Desktop\\list_liepin.csv"
column_list = ['岗位名称', '公司名称', '最低年薪', '最高年薪', '年薪单位', 
               '工作城市',  '工作区域', '学历要求', '工作年限', '语言要求', '年龄要求', '链接']
data = pd.DataFrame(columns=column_list)
data.to_csv(output_file, index=None, encoding="utf_8_sig", quoting=1)

chrome_options = Options()
chrome_options.add_argument('--disable-extensions')
chrome_options.add_argument('--profile-directory=Default')
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--disable-plugins-discovery");
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--headless")


for current_job_link in list_job_link:
    try:
        sub_html = urlopen(current_job_link)
        sub_bs = BeautifulSoup(sub_html.read().decode(sub_html.headers.get_content_charset()), 'html.parser')   
#         driver = webdriver.Chrome(options=chrome_options)
#         driver.implicitly_wait(5)
#         driver.get(current_job_link)
#         sub_bs = BeautifulSoup(driver.page_source, 'html.parser')
#         driver.close()

#         browser = webdriver.Firefox()
#         browser.get(current_job_link)
#         sub_bs = BeautifulSoup(browser.page_source, 'html.parser')
#         browser.close()
        
        
    except HTTPError as e:
        print("error: "+current_job_link)
        error_data = pd.DataFrame([current_job_link])
        error_data.to_csv("C:\\Users\\hao.ren3\\Desktop\\error_list.csv", index=None, header=None, mode='a')
        continue

    current_row = []

    title = find_first_element(sub_bs, 'h1', {}, 'text')
    current_row.append(title)

    company = find_first_element(sub_bs, 'a', {'data-promid':re.compile('^$')}, 'text')
    current_row.append(company)

    salary = find_first_element(sub_bs, 'p', {'class':'job-item-title'}, 'text')
    salary_min = find_salary(salary)
    current_row.append(salary_min)

    salary_max = find_salary(salary, min_value=False)
    current_row.append(salary_max)

    salary_unit = find_salary(salary, unit=True)
    current_row.append(salary_unit)

    location = find_first_element(sub_bs, 'p', {'class':'basic-infor'}, 'text', True)
    city_string = find_first_element(location, 'a', {}, 'text')
    city = find_place(city_string)
    current_row.append(city)

    district = find_place(city_string, city=False)
    current_row.append(district)

    div_profile = find_first_element(sub_bs, 'div', {'class':'job-qualifications'}, 'text', True)
    list_profile = div_profile.find_all('span', {})
    for current_profile in list_profile:
        current_row.append(current_profile.get_text())
    current_row.append(current_job_link)
    current_data = pd.DataFrame(np.reshape(current_row, (1, -1)))
    current_data.to_csv(output_file, index=None, header=None, encoding="utf_8_sig", quoting=1, mode='a')
#     delete_chromedriver()
    sleep(10)

In [146]:
test_data = pd.read_csv("C:\\Users\\hao.ren3\\Desktop\\error_list.csv", encoding = "ISO-8859-1")
test_data.iloc[:, 0]
list_test_data = test_data.values
list_visited_web = []
for i in list_test_data:
    list_visited_web.append(i[0])
list_visited_web

list_unvisited_web = []
for i in list_job_link:
    if i in list_visited_web:
        print("already visit "+i)
    else:
        list_unvisited_web.append(i)
        print("not visit "+i)

['https://www.liepin.com/job/1923223061.shtml',
 'https://www.liepin.com/job/1922955607.shtml',
 'https://www.liepin.com/job/1922553569.shtml',
 'https://www.liepin.com/job/1919064077.shtml',
 'https://www.liepin.com/job/1922738447.shtml',
 'https://www.liepin.com/job/1915395663.shtml',
 'https://www.liepin.com/job/1919525035.shtml',
 'https://www.liepin.com/job/1921488197.shtml',
 'https://www.liepin.com/job/1923225391.shtml',
 'https://www.liepin.com/job/1920970815.shtml',
 'https://www.liepin.com/job/1921183117.shtml',
 'https://www.liepin.com/job/1921183113.shtml',
 'https://www.liepin.com/job/1923224469.shtml',
 'https://www.liepin.com/job/1923096863.shtml',
 'https://www.liepin.com/job/1923096843.shtml',
 'https://www.liepin.com/job/1922616657.shtml',
 'https://www.liepin.com/job/1920344751.shtml',
 'https://www.liepin.com/job/1922641549.shtml',
 'https://www.liepin.com/job/1923223951.shtml',
 'https://www.liepin.com/job/1921619521.shtml',
 'https://www.liepin.com/job/1922025669.

In [4]:
test_url = "https://www.liepin.com/job/1922647641.shtml"
chrome_options = Options()
driver = webdriver.Chrome(options=chrome_options)
driver.get(test_url)
sub_bs = BeautifulSoup(driver.page_source, 'html.parser')
driver.close()

In [5]:
sub_bs

<html><head><link href="https://webapi.amap.com/theme/v1.3/style1536672475627.css" rel="stylesheet" type="text/css"/><script src="https://zz.bdstatic.com/linksubmit/push.js"></script><script src="//hm.baidu.com/hm.js?a2647413544f5a04f00da7eee0d5e200"></script>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>【结算岗招聘】京东物流招聘信息-猎聘</title>
<meta content="结算岗招聘,京东物流招聘信息" name="keywords"/>
<meta content="【京东物流招聘信息】诚聘【结算岗】年薪5-10万,北京京东尚科信息技术有限公司公司规模10000人以上,经验:1年以上经验,学历: 大专及以上,猎聘祝您顺利获得京东物流结算岗职位." name="description"/>
<link href="https://m.liepin.com/job/1922647641.shtml" media="only screen and (max-width: 640px)" rel="alternate"/>
<link href="https://www.liepin.com/job/1922647641.shtml" rel="canonical"/>
<meta content="province=成都;city=成都" name="location"/>
<meta content="pc" name="applicable-device"/>
<meta content="format=html5;url=https://m.liepin.com/job/1922647641.shtml" name="mobile-agent"/>
<meta content="no-transform" http-equiv="Cache-Control"/>
<meta content

In [205]:
from selenium import webdriver

url = 'https://hacpai.com/article/1566358988236'



'<html><head><link rel="stylesheet" type="text/css" href="https://webapi.amap.com/theme/v1.3/style1536672475627.css"><script src="https://zz.bdstatic.com/linksubmit/push.js"></script><script src="//hm.baidu.com/hm.js?a2647413544f5a04f00da7eee0d5e200"></script>\n    <meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n    <title>【结算岗招聘】京东物流招聘信息-猎聘</title>\n<meta name="keywords" content="结算岗招聘,京东物流招聘信息">\n<meta name="description" content="【京东物流招聘信息】诚聘【结算岗】年薪5-10万,北京京东尚科信息技术有限公司公司规模10000人以上,经验:1年以上经验,学历: 大专及以上,猎聘祝您顺利获得京东物流结算岗职位.">\n<link rel="alternate" media="only screen and (max-width: 640px)" href="https://m.liepin.com/job/1922647641.shtml">\n<link rel="canonical" href="https://www.liepin.com/job/1922647641.shtml">\n<meta name="location" content="province=成都;city=成都">\n<meta name="applicable-device" content="pc">\n<meta name="mobile-agent" content="format=html5;url=https://m.liepin.com/job/1922647641.shtml">\n<meta http-equiv="Cache-Control" content="no-transform">\n<me